# Quantum Fourier Transform (QFT) Tensor Network Analysis

This notebook demonstrates the creation, sequential contraction, and parallel contraction of a QFT tensor network using various algorithms such as Girvan–Newman and FlowCutter.

---



## Step 0: Loading software

 First of all, we create a new project to load all the neccesary software

In [1]:
] activate New_Project_on_QXTools;

  Activating project at `C:\Users\Usuario\OneDrive\Escriptori\anaconda_blogs\agost_2023\Novembre_desembre_2024\GitHub_resum_article_Costa_Ballena\New_Project_on_QXTools`


In [ ]:
import Pkg; 
Pkg.add("QXTools")
Pkg.add("QXGraphDecompositions")
Pkg.add("QXZoo")
Pkg.add("DataStructures")
Pkg.add("QXTns")
Pkg.add("NDTensors")
Pkg.add("ITensors")
Pkg.add("LightGraphs")
Pkg.add("PyCall")




    Updating registry at `C:\Users\Usuario\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Usuario\OneDrive\Escriptori\anaconda_blogs\agost_2023\Novembre_desembre_2024\GitHub_resum_article_Costa_Ballena\New_Project_on_QXTools\Project.toml`
  No Changes to `C:\Users\Usuario\OneDrive\Escriptori\anaconda_blogs\agost_2023\Novembre_desembre_2024\GitHub_resum_article_Costa_Ballena\New_Project_on_QXTools\Manifest.toml`


In [2]:
using QXTools
using QXTns
using QXZoo
using PyCall
using QXGraphDecompositions
using LightGraphs
using DataStructures
using TimerOutputs
using ITensors
using LinearAlgebra
using NDTensors


[ Info: OMEinsum loaded the CUDA module successfully


In [3]:
# Load custom functions
include("../src/funcions_article.jl");

## Step 1: Create a Quantum Fourier Transform Circuit
We begin by creating a QFT circuit based on the user-defined number of qubits.

In [4]:
# --- Step 1: Quantum Fourier Transform (QFT) Circuit Creation ---
@info "How many qubits do you want for the QFT circuit?"
n = parse(Int64, readline())
println("The QFT circuit created has $n qubits.")

# Create QFT circuit
cct = create_qft_circuit_bis(n)

tnc = convert_to_tnc(cct)  # Convert the QFT circuit into a tensor network circuit


[ Info: How many qubits do you want for the QFT circuit?


stdin>  20


The QFT circuit created has 20 qubits.


TensorNetworkCircuit(qubits => 20, gates => 464)

---

## Step 2: Sequential Contraction Using Girvan-Newman
We perform sequential contraction of the tensor network using the Girvan–Newman algorithm to get a contraction order.

In [5]:
# --- Step 2: Sequential Contraction using Girvan-Newman ---
s1 = Calcul_GN_Sequencial(cct, true)  # Perform sequential contraction
println("Sequential contraction result: ", s1)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            42.1s /  98.8%           2.33GiB /  99.2%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
3T. Final contraction      1    39.2s   94.3%   39.2s   2.27GiB   98.0%  2.27GiB
2T. Getting GN plan        1    2.00s    4.8%   2.00s   31.1MiB    1.3%  31.1MiB
1T. Obtaining a li...      1    357ms    0.9%   357ms   15.1MiB    0.6%  15.1MiB
────────────────────────────────────────────────────────────────────────────────
Sequential contraction result: fill(0.0009765624999999416 + 6.596081874690899e-19im)


---

## Step 3: Compare Results Using Another Contraction Plan
Here, we utilize FlowCutter to generate an alternative contraction plan and compare results.

In [6]:
# --- Step 3: Compare with Another Plan ---
using QXTools
using QXTools.Circuits

# Convert the QFT circuit into a tensor network circuit
tnc = convert_to_tnc(cct)  

# Find a good contraction plan using FlowCutter
plan = flow_cutter_contraction_plan(tnc; time=10)

num_qubits = cct.num_qubits

# Output states
outputs = ["0" ^ num_qubits, "1" ^ num_qubits, "1" ^ (num_qubits - 2) * "01"]
eixida = outputs[1]  # Select the first output as the target

# Evaluate the probability amplitude for different outputs

@show QXTools.single_amplitude(tnc, plan, eixida)
eixida = outputs[2]  # Select the second output
@show QXTools.single_amplitude(tnc, plan, eixida)
eixida = outputs[3]  # Select the third output
@show QXTools.single_amplitude(tnc, plan, eixida)

# Perform tensor network contraction using the plan
s = contract_tn!(tnc.tn, plan)
println("Contraction result using FlowCutter plan: ", s)

# Compare results
println("Do the results match? ", s1 ≈ s)

QXTools.single_amplitude(tnc, plan, eixida) = 0.0009765624999999408 + 1.2304869958639009e-18im
QXTools.single_amplitude(tnc, plan, eixida) = 0.0009765624999998817 + 1.0968001332144409e-20im
QXTools.single_amplitude(tnc, plan, eixida) = 0.0009765624999998878 + 9.279477547741198e-20im
Contraction result using FlowCutter plan: fill(0.0009765624999999408 + 1.2304869958639009e-18im)
Do the results match? true


---

## Step 4: Parallel Contraction Using ComPar
Finally, we use ComPar algorithms for parallel contraction.

In [7]:
# --- Step 4: Parallel Contraction using ComPar ---
# Define input and output states
num_qubits = cct.num_qubits
entrada = "0" ^ num_qubits

# Output states
outputs = ["0" ^ num_qubits, "1" ^ num_qubits, "1" ^ (num_qubits - 2) * "01"]
eixida = outputs[1]  # Select the first output as the target

n_com = 8  # Number of communities for the contraction
println("Using $n_com communities for contraction.")



Using 8 communities for contraction.


In [8]:
# Perform contraction using ComParCPU
s2 = ComParCPU(cct, entrada, eixida, n_com; timings=true, decompose=true)
println("Contraction result using ComParCPU: ", s)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            6.61s / 100.0%            500MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
3T.Final Contraction       1    3.94s   59.5%   3.94s    290MiB   57.9%   290MiB
1T.Obtaining Commu...      1    1.42s   21.5%   1.42s   24.9MiB    5.0%  24.9MiB
2T.Parallel contra...      1    1.26s   19.0%   1.26s    185MiB   37.1%   185MiB
────────────────────────────────────────────────────────────────────────────────
Contraction result using ComParCPU: fill(0.0009765624999999408 + 1.2304869958639009e-18im)


In [10]:
# Perform contraction using ComParCPU_para
s_para = ComParCPU_para(cct, entrada, eixida, n_com; timings=true, decompose=true)
println("Contraction result using ComParCPU_para: ", s_para)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            1.74s / 100.0%            332MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
1T.Obtaining Commu...      1    808ms   46.5%   808ms   7.73MiB    2.3%  7.73MiB
2T.Parallel contra...      1    593ms   34.1%   593ms    170MiB   51.2%   170MiB
3T.Final contracti...      1    336ms   19.3%   336ms    154MiB   46.5%   154MiB
────────────────────────────────────────────────────────────────────────────────
Contraction result using ComParCPU_para: fill(0.0009765624999999416 + 6.596081874690899e-19im)


In [11]:
# Compare results
println("Do all the results match? ", s1 ≈ s≈ s2 ≈ s_para)

Do all the results match? true


---

### Summary
This notebook demonstrated:
1. The creation of a QFT tensor network.
2. Sequential contraction using the Girvan–Newman algorithm.
3. Alternative contraction using FlowCutter.
4. Parallel contraction with ComParCPU and ComParCPU_para.

Thank you for exploring tensor network contraction with us!